In [20]:
import pandas as pd
import os


In [21]:
orders = pd.read_csv("/E2E Data Analysis Project/data/raw/olist_orders_dataset.csv")
order_items = pd.read_csv("/E2E Data Analysis Project/data/raw/olist_order_items_dataset.csv")
products = pd.read_csv("/E2E Data Analysis Project/data/raw/olist_products_dataset.csv")
customers = pd.read_csv("/E2E Data Analysis Project/data/raw/olist_customers_dataset.csv")
payments = pd.read_csv("/E2E Data Analysis Project/data/raw/olist_order_payments_dataset.csv")

In [22]:
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"], errors="coerce")


In [23]:
orders_small = orders[
    [
        "order_id",
        "customer_id",
        "order_status",
        "order_purchase_timestamp",
    ]
].copy()

order_items_small = order_items[
    [
        "order_id",
        "order_item_id",
        "product_id",
        "seller_id",
        "price",
        "freight_value",
    ]
].copy()

products_small = products[
    [
        "product_id",
        "product_category_name",
    ]
].copy()

customers_small = customers[
    [
        "customer_id",
        "customer_unique_id",
        "customer_state",
    ]
].copy()


In [24]:
payments_agg = (
    payments.groupby("order_id", as_index=False)
    .agg(
        payment_value_total=("payment_value", "sum"),
        payment_installments_max=("payment_installments", "max"),
        payment_type_nunique=("payment_type", "nunique"),
    )
)


In [25]:
payment_type_mode = (
    payments.groupby("order_id")["payment_type"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    .reset_index()
    .rename(columns={"payment_type": "payment_type_primary"})
)

payments_agg = payments_agg.merge(payment_type_mode, on="order_id", how="left")
payments_agg.head()


,order_id,payment_value_total,payment_installments_max,payment_type_nunique,payment_type_primary
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,2,1,credit_card
1,00018f77f2f0320c557190d7a144bdd3,259.83,3,1,credit_card
2,000229ec398224ef6ca0657da4fc703e,216.87,5,1,credit_card
3,00024acbcdf0a6daa1e931b038114c75,25.78,2,1,credit_card
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,3,1,credit_card


In [26]:
fact = order_items_small.merge(orders_small, on="order_id", how="left", validate="many_to_one")


In [27]:
fact.shape, order_items_small.shape


((112650, 9), (112650, 6))

In [28]:
fact = fact.merge(products_small, on="product_id", how="left", validate="many_to_one")


In [29]:
fact = fact.merge(customers_small, on="customer_id", how="left", validate="many_to_one")


In [30]:
fact = fact.merge(payments_agg, on="order_id", how="left", validate="many_to_one")


In [31]:
fact["purchase_month"] = fact["order_purchase_timestamp"].dt.to_period("M").astype(str)


In [32]:
fact["item_revenue"] = fact["price"]
fact["item_total_value"] = fact["price"] + fact["freight_value"]


In [33]:
fact.isna().mean().sort_values(ascending=False).head(10)


product_category_name       0.014230
payment_type_primary        0.000027
payment_type_nunique        0.000027
payment_installments_max    0.000027
payment_value_total         0.000027
customer_unique_id          0.000000
item_revenue                0.000000
purchase_month              0.000000
customer_state              0.000000
order_id                    0.000000
dtype: float64

In [34]:
fact.duplicated(subset=["order_id", "order_item_id"]).sum()


0

In [35]:
os.makedirs("data/processed", exist_ok=True)
fact.to_parquet("/E2E Data Analysis Project/data/processed/fact_order_items.parquet", index=False)


In [36]:
fact["order_status"].value_counts().head()

delivered     110197
shipped         1185
canceled         542
invoiced         359
processing       357
Name: order_status, dtype: int64

In [37]:
fact["payment_value_total"].isna().mean()


2.663115845539281e-05